In [5]:
import sys 
import os
sys.path.append(os.path.abspath("/Users/vachananand/Documents/algos"))
import trading_api
import pandas as pd
import time
import numpy as np
import os.path
import time
from binance.client import Client
from datetime import timedelta, datetime
from decimal import Decimal

In [6]:
class Binance_FutueData:
    def __init__(self,key,secret):
        self.key = key
        self.secret = secret
        self.binance_client = Client(api_key=key, api_secret=secret)
        self.cryptos = None
        self.timeframe = None
        self.start_date = None
        self.end_data = None
        self.data = dict()
    
    
    def get_time_period(self,symbol,timeframe,start,end):
        start_date = int(datetime.timestamp(datetime.strptime(start, '%d %b %Y')))*1000
        if end :
            end_date = int(datetime.timestamp(datetime.strptime(end, '%d %b %Y')))*1000
        else:
            end_date = int(self.binance_client.futures_klines(symbol=symbol, interval = timeframe)[-1][0])
        return start_date,end_date
    
    def get_historic_ohlcv(self,timeframe,start,end):
        for symbol in self.cryptos:
            try:
                ohlcv_data = dict()
                ohlcv_data["time"] = list()
                ohlcv_data["open"] = list()
                ohlcv_data["high"] = list()
                ohlcv_data["low"] = list()
                ohlcv_data["close"] = list()
                ohlcv_data["volume"] = list()
                start_date,end_date = self.get_time_period(symbol,timeframe,start,end)
                print("Getting historic data for",symbol,"... ")
                ohlcv = self.binance_client.futures_klines(symbol = symbol,interval = timeframe,startTime =start_date,endTime = end_date )
                while ohlcv[-1][0] != end_date:
                    new_ohlcv = self.binance_client.futures_klines(symbol = symbol,interval = timeframe,startTime = ohlcv[-1][0],endTime = end_date)
                    ohlcv += new_ohlcv
                    time.sleep(0.05)
                for each_candle in ohlcv[:-1]:
                    ohlcv_data["time"].append(Decimal(each_candle[0]))
                    ohlcv_data["open"].append(Decimal(each_candle[1]))
                    ohlcv_data["high"].append(Decimal(each_candle[2]))
                    ohlcv_data["low"].append(Decimal(each_candle[3]))
                    ohlcv_data["close"].append(Decimal(each_candle[4]))
                    ohlcv_data["volume"].append(Decimal(each_candle[5]))
                self.data[symbol] = pd.DataFrame(ohlcv_data)
            except Exception as e:
                print("CANNOT GET DATA FOR :",symbol)
                print(e)
    
    def set_symbol(self,cryptos):
        if cryptos == "ALL":
            symbols = list()
            all_tickers = self.binance_client.futures_mark_price()
            for ticker in all_tickers:
                symbols.append(ticker["symbol"])
            self.cryptos = symbols
        else:
            self.cryptos = cryptos
            
    
    def get_data(self,cryptos,timeframe,start_date,end_date = None):
        self.set_symbol(cryptos)
        self.get_historic_ohlcv(timeframe,start_date,end_date)
        
        



In [7]:
data_ob = Binance_FutueData(trading_api.binance["oz07"]["key"],trading_api.binance["oz07"]["secret"])

In [8]:
data_ob.get_data("ALL","1d","1 OCT 2020")

Getting historic data for EOSUSDT ... 
Getting historic data for SUSHIUSDT ... 
Getting historic data for BALUSDT ... 
CANNOT GET DATA FOR : BTTUSDT
APIError(code=-1121): Invalid symbol.
Getting historic data for KNCUSDT ... 
Getting historic data for SRMUSDT ... 
Getting historic data for ENJUSDT ... 
Getting historic data for ZRXUSDT ... 
Getting historic data for QTUMUSDT ... 
Getting historic data for ATOMUSDT ... 
Getting historic data for IOTAUSDT ... 
Getting historic data for WAVESUSDT ... 
Getting historic data for ADAUSDT ... 
Getting historic data for NEARUSDT ... 
Getting historic data for BCHUSDT ... 
Getting historic data for XTZUSDT ... 
Getting historic data for BNBUSDT ... 
Getting historic data for IOSTUSDT ... 
Getting historic data for HNTUSDT ... 
Getting historic data for ETCUSDT ... 
Getting historic data for XMRUSDT ... 
Getting historic data for YFIUSDT ... 
Getting historic data for ZECUSDT ... 
Getting historic data for ETHUSDT ... 
Getting historic data for 

In [9]:
data_ob.data["EOSUSDT"]

,time,open,high,low,close,volume
0,1601510400000,2.579,2.627,2.484,2.545,12011756.4
1,1601596800000,2.545,2.558,2.412,2.479,13709561.6
2,1601683200000,2.479,2.503,2.467,2.477,4486738.9
3,1601769600000,2.477,2.536,2.464,2.518,6337450.5
4,1601856000000,2.517,2.539,2.491,2.526,6437534.3
5,1601942400000,2.527,2.973,2.485,2.696,106360686.1
6,1602028800000,2.696,2.718,2.588,2.643,43141088.5
7,1602115200000,2.643,2.654,2.485,2.607,22288490.3
8,1602201600000,2.606,2.657,2.580,2.642,12946941.5
9,1602288000000,2.641,2.721,2.634,2.643,13267578.6


In [14]:
symbols = ["ADAUSDT","OMGUSDT"]
data_ob.get_data(symbols,"1d","1 JAN 2020")

Getting historic data for ADAUSDT ... 
Getting historic data for OMGUSDT ... 


In [16]:
data_ob.data["ADAUSDT"]

,time,open,high,low,close,volume
0,1580428800000,0.05450,0.05559,0.05209,0.05387,244632854
1,1580515200000,0.05383,0.05745,0.05304,0.05620,372594843
2,1580601600000,0.05616,0.05960,0.05484,0.05583,481355438
3,1580688000000,0.05577,0.05808,0.05526,0.05718,337457383
4,1580774400000,0.05718,0.05789,0.05406,0.05601,267287553
...,...,...,...,...,...,...
264,1603238400000,0.10237,0.10875,0.10131,0.10597,457541286
265,1603324800000,0.10591,0.11300,0.10530,0.11089,485211197
266,1603411200000,0.11085,0.11249,0.10521,0.10814,456747054
267,1603497600000,0.10811,0.11009,0.10740,0.10813,235916885
